<a href="https://colab.research.google.com/github/alitoast/Exam_IR/blob/fetcher_parser-dev/F_P_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Librerie#

In [26]:
#url di prova
# ha sitemap-index ma non priority etc.
start_url_uno = "https://www.dragopublisher.com/it/"
# non ha sitemap
start_url_due = "https://www.wildraccoon.it/shop/"
# sito giocattolo ha tutto
start_url = "https://nonciclopedia.org/wiki/Wikipedia" #--> no è in italiano
start_url= "https://www.vice.com/en/contributor/vice-uk/"

In [2]:
!pip install rfc3986

In [3]:
import pandas as pd
import re
from urllib.parse import urlparse
import urllib.robotparser #per gestire il file robot.txt
import xml.etree.ElementTree as ET  #per gestire i file xml
import requests #per http
import time #per gestire il tempo
from bs4 import BeautifulSoup #per gestire il parsing del html (si può usare anche per xml)
import rfc3986 # per la normalizzazione degli urls


#Fetcher#
gestione richieste e download pagine web

In [4]:
class UserAgentPolicy:
  '''
     Class to identify user agents and store relevant information to manage their requests appropriately over time
  '''
  def __init__(self, name, base_url, path_disallow, crawl_delay, request_rate,last_access, header):
        self.base_url = base_url
        self.name = name
        self.path_disallow = path_disallow
        self.crawl_delay = crawl_delay
        self.request_rate = request_rate
        self.last_access = last_access
        self.header = header

In [5]:
# Useragents
useragent_dict = {"Googlebot": UserAgentPolicy("Googlebot",None, None, None, None, 1e9,'Mozilla/5.0 (compatible; Googlebot/2.1; +http://www.google.com/bot.html)'),
                  "Bingbot": UserAgentPolicy("Bingbot",None, None, None, None, 1e9,'Mozilla/5.0 (compatible; Bingbot/2.0; +http://www.bing.com/bingbot.htm)'),
                  "Slurp": UserAgentPolicy("Slurp",None, None, None, None, 1e9, 'Mozilla/5.0 (compatible; Yahoo! Slurp; http://help.yahoo.com/help/us/ysearch/slurp)'),
                  "DuckDuckbot": UserAgentPolicy("DuckDuckbot",None, None, None, None, 1e9,'DuckDuckBot/1.0; (+http://duckduckgo.com/duckduckbot.html)' ),
                  "Yandex": UserAgentPolicy("Yandex",None, None, None, None, 1e9,'Mozilla/5.0 (compatible; YandexBot/3.0; +http://yandex.com/bots)' ),
                  "*": UserAgentPolicy("*",None, None, None, None, 1e9,'Mozilla/5.0 (compatible; MyBot/1.0; +http://example.com/bot)' )
                  }
default_agent = useragent_dict["Googlebot"]

# header generated by ChatGPT
useragent_dict_header = {'Googlebot':'Mozilla/5.0 (compatible; Googlebot/2.1; +http://www.google.com/bot.html)',
                    'Bingbot':'Mozilla/5.0 (compatible; Bingbot/2.0; +http://www.bing.com/bingbot.htm)',
                    'Slurp':'Mozilla/5.0 (compatible; Yahoo! Slurp; http://help.yahoo.com/help/us/ysearch/slurp)',
                    'DuckDuckbot':'DuckDuckBot/1.0; (+http://duckduckgo.com/duckduckbot.html)',
                    'Yandex':'Mozilla/5.0 (compatible; YandexBot/3.0; +http://yandex.com/bots)',
                    '*': 'Mozilla/5.0 (compatible; MyBot/1.0; +http://example.com/bot)'}

In [6]:
# global variable to keep track of the visited site
visulized_url = set()

In [28]:
def check_robots(url,useragent=default_agent):

  '''
    Inputs:
        url (str): The base URL of the website to analyze.
        useragent (class): The user agent to use when checking access permissions. Default is '*'.

    Output:
        sitemap (list or None): List of sitemap URLs found in robots.txt, if any.

    Description:
        This function checks the robots.txt file of a given website to determine:
        - Whether the given user agent is allowed to fetch content.
        - Which paths are disallowed for the user agent.
        - The crawl delay and request rate for the user agent.
        - Any sitemap URLs provided in the robots.txt.

        It updates the given useragent object with these details.
    '''

  useragent.base_url = url

  # Initialize the robot parser
  rfl = urllib.robotparser.RobotFileParser()
  # Construct the full URL to the robots.txt file
  new_url = url + "/robots.txt"

  rfl.set_url(new_url)
  # Read and parse the robots.txt file
  rfl.read()
  check = rfl.can_fetch(useragent.name,new_url)
  if check:
    print("{} lo user può effettuare il fetch dei dati".format(check))
  else:
    print("lo user non può effettuare il fetch dei dati")

  # Retrieve the list of the path not to follow
  path_disallow = rfl.parse("Disallow")
  useragent.path_disallow = path_disallow

  # Retrieve the crawl delay (minimum interval between requests)
  crawl_delay = rfl.crawl_delay(useragent)
  useragent.crawl_delay = crawl_delay

  # Retrieve the request rate (tuple of requests per time interval)
  request_rate = rfl.request_rate(useragent)
  useragent.request_rate = request_rate

  # Retrieve any sitemap URLs declared in the robots.txt
  sitemap = rfl.site_maps()
  print("sitemap: {}".format(sitemap))

  return sitemap

In [8]:
def check_time(useragent=default_agent):

  '''
    Inputs:
        useragent (UserAgentPolicy): The user agent object containing rate-limiting information.

    Output:
      None

    Description:
        Ensures that the user agent respects the specified crawl delay and request rate policies:
        - Calculates the appropriate delay based on the more restrictive value between `crawl_delay` and `request_rate`.
        - If neither is defined, defaults to a delay of 1.5 seconds.
        - Ensures the delay is respected by pausing execution as needed.
        - Updates `last_access` to the current time after waiting.
  '''


  # If both crawl_delay and request_rate are defined, use the more restrictive one (converted to nanoseconds)

  if useragent.request_rate and useragent.crawl_delay:
     # Calculate delay from request rate: total interval (in ns) divided by number of requests
     request_delay = (useragent.request_rate[1] * 1e9) / useragent.request_rate[0]
     # Convert crawl delay from seconds to nanoseconds
     crawl_delay = useragent.crawl_delay * 1e9
     # Use the larger (i.e., more restrictive) delay
     delay = max(request_delay, crawl_delay)

  elif useragent.crawl_delay:
     delay = useragent.crawl_delay * 1e9

  elif useragent.request_rate:
     delay = (useragent.request_rate[1] * 1e9) / useragent.request_rate[0]

  # If neither crawl_delay and request rate are defined, use a default delay of 1.5 seconds
  else:
    delay = 1.5 * 1e9  # 1.5 secondi in nanosecondi

  # Get the current time in nanoseconds
  now = time.monotonic_ns()
  wait = max(0, (useragent.last_access + delay) - now)

  if wait > 0:
     time.sleep(wait / 1e9)

  # Update last_access to the current time to enforce timing on next request
  useragent.last_access = time.monotonic_ns()


In [9]:
def fetch(url,useragent=default_agent):
  '''
    Inputs:
        url (str): The target URL to fetch.
        useragent (UserAgentPolicy): The user agent object containing headers and rate-limiting rules.

    Outputs:
        str or None: The HTML content of the page if the request is successful (HTTP 200), otherwise None.

    Description:
        Sends an HTTP GET request to the specified URL, respecting the request policies defined for the given user agent.

    '''

  # Check if the page is been visited alredy
  if url not in visulized_url:

    # Enforce crawl delay and request rate restrictions for the user agent
    check_time(useragent)

    # Build the request headers using the specified user agent (optional, useful for testing)
    headers = {
          'User-Agent': useragent.header
     }

    # Send the HTTP GET request
    try:
       response = requests.get(url, headers=headers, timeout=10)
       if response.status_code == 200:
         print("Pagina recuperata con successo:{}".format(url))
         html = response.text
         visulized_url.add(url)
         return html
       else:
        print("Errore nella ricezione del sito:{}".format(url))
        return None

    # Handle any request-related exceptions (connection errors, timeouts, etc.)
    except requests.RequestException as e:
          print("Eccezione durante la richiesta:", e)
          return None
  else:
    print("Pagina già visulizzata:{}".format(url))
    return None

# Parser#
analisi contenuto della pagina, estrazione testo e link e normalizzazione link trovati

In [10]:
def normalize_url(url):

  '''
    Input:
        url (str): The URL to be normalized.

    Output:
        str: The normalized URL as a string. If normalization fails, the original URL is returned.

    Description:
        Normalizes the input URL according to the RFC 3986 standard and returns a string representation.
        This function uses the `rfc3986` library to parse and normalize the URL according to the
        URI standard defined in RFC 3986. This includes handling issues such as case normalization,
        removing default ports, sorting query parameters (if applicable), and more.

        If an error occurs during normalization (e.g., invalid input), it catches the exception
        and returns the original URL as a fallback.
  '''

  try:
      uri = rfc3986.uri_reference(url).normalize()
      return uri.unsplit()

  except Exception as e:
      print(f"Errore nella normalizzazione URL: {url} – {e}")
      return url


In [11]:
def check_spider_traps(url):

  '''

    Input:
     url(str):The URL to be analyzed.

    Output:
      - Returns False if the URL is suspicious or considered a "spider trap."
      - Returns True if the URL seems safe to crawl.

    Description:
      Checks whether a given URL is potentially harmful or could trap a web crawler
      in infinite loops or unnecessary crawling paths.

  '''

  MAX_URL_LENGTH = 200   # Arbitrary maximum allowed URL length
  MAX_PATH_DEPTH = 6     # Maximum allowed number of slashes in path
  trap_pattern = re.compile(r"(calendar|sessionid|track|ref|sort|date=|page=\d{3,})", re.IGNORECASE)   # Pattern matching common signs of spider traps:
                                                                                                       # calendars, session IDs, tracking params, endless pagination, etc.

  link = urlparse(url)

  # Check URL length
  if len(link) > MAX_URL_LENGTH:
     return False

  # Check path depth (number of '/' in path)
  if link.path.count('/') > MAX_PATH_DEPTH:
     return False

  # Check for suspicious patterns in the URL
  if trap_pattern.search(url):
     return False

  return True


In [12]:

def parse_sitemap(sitemap_list):

    '''
      Input:
        sitemap_list (list): A list of sitemap URLs to parse.

      Output:
          Returns a DataFrame containing:
         - URL
         - Priority
         - Update frequency

      Description:
         Supports both XML and HTML sitemaps. Handles nested sitemap indexes.
         Returns a DataFrame with the parsed sitemap entries.

    '''
    namespace = {'ns': 'http://www.sitemaps.org/schemas/sitemap/0.9'}
    all_entries = []

    if not sitemap_list:
        print("Nessuna sitemap fornita.")
        return pd.DataFrame(columns=['url', 'priority', 'update'])

    def parse_single_sitemap(sitemap_url):
        try:
            response = requests.get(sitemap_url, timeout=10)
            response.raise_for_status()
        except requests.RequestException as e:
            print(f"Errore durante il recupero di {sitemap_url}: {e}")
            return []

        content_type = response.headers.get("Content-Type", "").lower()

        # Sitemap XML
        if "xml" in content_type:
            try:
                root = ET.fromstring(response.content)
            except ET.ParseError as e:
                print(f"Errore nel parsing XML: {e}")
                return []

            # If it's a sitemap index, recursively parse child sitemaps
            if root.tag.endswith('index'):
                sitemap_urls = [loc.text for loc in root.findall('ns:sitemap/ns:loc', namespace)]
                entries = []
                for sub_url in sitemap_urls:
                    entries.extend(parse_single_sitemap(sub_url))
                return entries

            #  # If it's a standard URL set
            elif root.tag.endswith('urlset'):
                return [{
                    'url': url.findtext('ns:loc', default='', namespaces=namespace),
                    'priority': url.findtext('ns:priority', default=None, namespaces=namespace),
                    'update': url.findtext('ns:changefreq', default=None, namespaces=namespace)
                } for url in root.findall('ns:url', namespace)]

        # Handle HTML sitemaps (fallback)
        elif "html" in content_type:
            soup = BeautifulSoup(response.content, "html.parser")
            return [{
                'url': normalize_url(a['href']),
                'priority': None,
                'update': None
            } for a in soup.find_all('a', href=True)]

        else:
            print(f"Formato non riconosciuto per {sitemap_url} ({content_type})")
            return []

    #  Process each sitemap URL in the list
    for sitemap_url in sitemap_list:
        if sitemap_url:
            entries = parse_single_sitemap(sitemap_url)
            for entry in entries:
                url = normalize_url(entry['url']) #normalizzo l'url
                if check_spider_traps(url): # provo a limitare le spider-trap
                    entry['url'] = url

            all_entries.extend(entries)

    # Create a DataFrame and drop duplicates
    df = pd.DataFrame(all_entries, columns=['url', 'priority', 'update']).drop_duplicates()

    return df


In [13]:
def parse_page_url(html,sitemaps_urls,useragent=default_agent):

  '''
     Inputs:
     - html(str): The HTML content of the page.
     - sitemaps_urls(str): A list of URLs already known from sitemaps.
     - useragent(class): A user agent object containing disallowed paths (robots.txt rules).

     Output:
     - A list of new, allowed URLs extracted from the page.

     Description:
       Given an HTML page, this function extracts all URLs and filters them based on:
       - Exclusion of private or disallowed directories (from robots.txt rules)
       - Deduplication with URLs already found in the sitemap
       - Optional normalization and trap filtering

  '''

  # Parse HTML content
  soup = BeautifulSoup(html, 'html.parser')

  urls = []
  # Extract URLs from href attributes (e.g., <a>, <link>, etc.)
  for tag in soup.find_all(href=True):
      urls.append(tag['href'])

  # Extract URLs from src attributes (e.g., <img>, <script>, etc.)
  for tag in soup.find_all(src=True):
      urls.append(tag['src'])

  # Filter out disallowed URLs based on robots.txt rules
  if useragent.path_disallow != None:
    for url in urls:
      for path in useragent.path_disallow:
        if path in url:
         urls.remove(url)

  # Normalize URLs (e.g., remove fragments, resolve relative paths, etc.)
    urls = [normalize_url(url) for url in urls]

  # Remove duplicate URLs
    urls = list(set(urls))

  # Filter out potential spider traps
    urls = [url for url in urls if check_spider_traps(url)]

  # Filter out URLs that are already in the sitemap
  if sitemaps_urls != None:
    def_urls = sitemaps_urls
    for url in urls:
      if url not in def_urls:
         def_urls.append(url)
  else:
    def_urls = urls

  return def_urls



In [20]:
def parse_page_tags_all(html,tags_type = None):

  '''
    Inputs:
    - html: The HTML content as a string.
    - tags_type: A list of tag names to search for (default includes common content tags).

    Output:
    - A list of text strings extracted from the specified tags, preserving DOM order

    Description:
      Parses the given HTML content and extracts text from specified HTML tags.

  '''
  if tags_type == None:
     tags_type = ['h1', 'h2', 'h3', 'h4', 'h5', 'h6', 'p', 'div', 'span', 'a']

  # Initialize the HTML parser
  soup = BeautifulSoup(html, "html.parser")

  # Find tag to remove
  for tag in soup(['script','style','footer','nav','noscript','header','form','aside']):
      tag.decompose()

  # Find all comments and removes them
  for comment in soup.find_all(string=lambda text: isinstance(text,comment)):
      comment.extract()

  # Find all tags of the specified types (respects DOM order)
  tags = soup.find_all(tags_type)

  # Extract clean text from each tag (removing whitespace and combining with spaces)
  texts = [tag.get_text(separator=' ', strip=True) for tag in tags]

  return texts



In [16]:
html = fetch(start_url_due, default_agent)
words = parse_page_tags_all(html)
print(words)
print(default_agent.last_access)

Pagina recuperata con successo:https://www.wildraccoon.it/shop/
['Home', 'Le birre', 'Shop Lattine Merchandising', 'Lattine', 'Merchandising', 'Il Metodo', 'Chi siamo', 'Bolli spina', 'Contatti', 'Home', 'Le birre', 'Shop Lattine Merchandising', 'Lattine', 'Merchandising', 'Il Metodo', 'Chi siamo', 'Bolli spina', 'Contatti', 'SHOP', 'Shop', 'Kopernik was a woman 24,00 € – 66,00 € Scegli Questo prodotto ha più varianti. Le opzioni possono essere scelte nella pagina del prodotto', 'Kopernik was a woman', 'Beyond Infinity 19,00 € – 54,00 € Scegli Questo prodotto ha più varianti. Le opzioni possono essere scelte nella pagina del prodotto', 'Beyond Infinity', 'Straight to harvest 22,00 € – 62,00 € Scegli Questo prodotto ha più varianti. Le opzioni possono essere scelte nella pagina del prodotto', 'Straight to harvest', 'Each others paranoia 20,00 € – 58,00 € Scegli Questo prodotto ha più varianti. Le opzioni possono essere scelte nella pagina del prodotto', 'Each others paranoia', 'Vfx 19,0

#Test#

sito privo di sitemap, provo con il parse a trovare i link

In [27]:
#default_agent = 'Googlebot'
sitemap = check_robots(start_url_due,default_agent)
html = fetch(start_url_due, default_agent)
if sitemap == None:
  urls = parse_page_url(html,None,default_agent)
print(urls)

True il sito puo effettuare il fetch dei dati
sitemap: None
Pagina già visulizzata:https://www.wildraccoon.it/shop/


TypeError: Incoming markup is of an invalid type: None. Markup must be a string, a bytestring, or an open filehandle.

In [ ]:
#useragent = '*'
sitemap = check_robots(start_url_due,useragent=useragent_dict['*'])
html = fetch(start_url_due, default_agent)
if sitemap == None:
  urls = parse_page_url(html,None,default_agent)
print(urls)

sito con sitemap provo ad anlizzarla

In [ ]:
#default_agent = 'Googlebot'
sitemap = check_robots(start_url,default_agent)
html = fetch(start_url_due, default_agent)
if sitemap != None:
  df = parse_sitemap(sitemap)
  print(df)

urls = parse_page_url(html,df['url'].tolist(),default_agent)
print(urls)

True lo user può effettuare il fetch dei dati
sitemap: ['https://www.vice.com/sitemap.xml']
Pagina già visulizzata:https://www.wildraccoon.it/shop/


In [ ]:
#default_agent = '*'
sitemap = check_robots(start_url_uno,useragent=useragent_dict['*'])
html = fetch(start_url_due, default_agent)
if sitemap != None:
  df = parse_sitemap(sitemap)
  print(df)

urls = parse_page_url(html,df['url'].tolist(),default_agent)
print(urls)